In [1]:
library("lme4")
library("ggplot2")
library("dplyr")
library(MASS)
library("BayesFactor")
library("car")
library("scales")
library("lmerTest")
#library("MuMIn")
library("plyr")
library("rstatix")
library("ggpubr")
library("knitr")
library("corrplot")
library("RColorBrewer")

Loading required package: Matrix


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select


Loading required package: coda

************
Welcome to BayesFactor 0.9.12-4.4. If you have questions, please contact Richard Morey (richarddmorey@gmail.com).

Type BFManual() to open the manual.
************

Loading required package: carData


Attaching package: 'car'


The following object is masked from 'package:dplyr':

    recode



Attaching package: 'lmerTest'


The following object is masked from 'package:lme4':

    lmer


The following object is masked from 'package:stats':

    step


------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functi

In [2]:
#Load the data => Here, we are filtering the cases in which subjects had a MuIFI of 0
FinalPath<-'./R_Data_ALL.txt'
data <- read.delim(FinalPath, sep="\t", header=T, stringsAsFactors=F)

In [3]:
fast_data <- data 
table(fast_data$Agent, fast_data$GameNb)

           
              1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
  Human_FFF  41  41  41  41  41  41  41  41  41  41  41  41  41  41  41
  MS17_FFF  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100

# Compute BIC for fast-speed data & baseline model

In [4]:
#RMSE function definition
RMSE = function(mod, obs){
  sqrt(mean((mod - obs)^2))
}

RSS = function(mod,obs){
    sum((mod - obs)^2)
}

BIC = function(RSS,n,k){
    n*log(RSS/n)+k*log(n)
}

In [5]:
#Filtered data set
head(fast_data,10)

,Type,Speed,Agent,subject,GameNb,Deflations,Misses,Resets,ShotReg,Entropy,LogCV,ShotPer,RightReg,RightPer,Score
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Human,FFF,Human_FFF,A11LNK1U3DT08V,1,23,45,17,0.5308518,3.117663,-1.045539,512,0.3869265,528,526
2,Human,FFF,Human_FFF,A11LNK1U3DT08V,2,15,39,9,0.4488193,3.063328,-1.257139,496,0.3037540,496,1626
3,Human,FFF,Human_FFF,A11LNK1U3DT08V,3,19,50,10,0.3585762,3.053468,-1.233135,480,0.2317211,496,1604
4,Human,FFF,Human_FFF,A11LNK1U3DT08V,4,11,33,11,0.4825653,2.692394,-1.325335,512,0.3582805,512,1640
5,Human,FFF,Human_FFF,A11LNK1U3DT08V,5,19,49,12,0.3362358,3.552236,-1.092696,432,0.1994396,432,1408
6,Human,FFF,Human_FFF,A11LNK1U3DT08V,6,12,31,4,0.6172684,2.654637,-1.510542,496,0.4029821,496,1640
7,Human,FFF,Human_FFF,A11LNK1U3DT08V,7,12,30,2,0.5474491,2.438955,-1.556296,496,0.3957245,496,1840
8,Human,FFF,Human_FFF,A11LNK1U3DT08V,8,18,30,7,0.4991873,2.795948,-1.378016,480,0.2724625,496,1844
9,Human,FFF,Human_FFF,A11LNK1U3DT08V,9,18,25,4,0.5047437,2.747487,-1.326809,496,0.3023007,496,1656


In [6]:
#1) Performance results
muTab<-aggregate(fast_data$Score, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameNb","Score")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$Score
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$Score
}

#RSS per condition
rss_perf <- RSS(M_vec,H_vec)
rss_perf

[1] 4247616

In [7]:
#2) Entropy results
fast_data <- data
fast_data <- fast_data %>% filter(!Entropy=='NaN')
muTab<-aggregate(fast_data$Entropy, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameNb","Entropy")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$Entropy
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$Entropy
}

#RSS per condition
rss_ent <- RSS(M_vec,H_vec)
rss_ent

[1] 1.385467

In [8]:
#3) Log CV ISI
fast_data <- data 
fast_data <- fast_data %>% filter(!LogCV=='NaN')
muTab<-aggregate(fast_data$LogCV, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","LogCV")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$LogCV
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$LogCV
}

#RSS per condition
rss_lcv <- RSS(M_vec,H_vec)
rss_lcv

[1] 1.994851

In [9]:
#4) Resets
fast_data <- data
muTab<-aggregate(fast_data$Resets, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Resets")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$Resets
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$Resets
}
#RSS per condition
rss_res <- RSS(M_vec,H_vec)
rss_res

[1] 1234.444

In [10]:
#5) Deflations
fast_data <- data
muTab<-aggregate(fast_data$Deflations, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Deflations")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$Deflations
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$Deflations
}

#RSS per condition
rss_def <- RSS(M_vec,H_vec)
rss_def

[1] 11972.3

In [11]:
#6) Misses
fast_data <- data
muTab<-aggregate(fast_data$Misses, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","Misses")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$Misses
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$Misses
}

#RSS per condition
rss_mis <- RSS(M_vec,H_vec)
rss_mis

[1] 21353.72

In [12]:
#7) Shot periodicity
fast_data <- data
fast_data <- fast_data %>% filter(!ShotPer=='NaN')
muTab<-aggregate(fast_data$ShotPer, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","ShotPer")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$ShotPer
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$ShotPer
}

#RSS per condition
rss_per <- RSS(M_vec,H_vec)
rss_per

[1] 39238.76

In [13]:
#8) Shot regularity
fast_data <- data
fast_data <- fast_data %>% filter(!ShotReg=='NaN')
muTab<-aggregate(fast_data$ShotReg, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","ShotReg")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$ShotReg
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$ShotReg
}

#RSS per condition
rss_reg <- RSS(M_vec,H_vec)
rss_reg

[1] 0.7764219

In [14]:
#9) Right regularity
fast_data <- data
fast_data <- fast_data %>% filter(!RightReg=='NaN')
muTab<-aggregate(fast_data$RightReg, by=list(fast_data$Agent,fast_data$GameNb), FUN=mean)

colnames(muTab)<-c("Agent","GameNb","RightReg")
H_vec <- c(1:15)
for (i in 1:15)
{
    H_vec[i] <- muTab[which(muTab$Agent=="Human_FFF" & muTab$GameNb==i),]$RightReg
}
M_vec <- c(1:15)
for (i in 1:15)
{
    M_vec[i] <- muTab[which(muTab$Agent=="MS17_FFF" & muTab$GameNb==i),]$RightReg
}

#RSS per condition
rss_Rreg <- RSS(M_vec,H_vec)
rss_Rreg

[1] 0.1932175

In [15]:
#BIC measure - Baseline model
n=15
k=0
Points_BIC <- BIC(rss_perf,n,k)
Points_BIC
Res_BIC <- BIC(rss_res,n,k)
Res_BIC
Def_BIC <- BIC(rss_def,n,k)
Def_BIC
Mis_BIC <- BIC(rss_mis,n,k)
Mis_BIC
print("variability")
Ent_BIC <- BIC(rss_ent,n,k)
Ent_BIC
lCV_BIC <- BIC(rss_lcv,n,k)
lCV_BIC
print("ACF measures")
Per_BIC <- BIC(rss_per,n,k)
Per_BIC
Reg_BIC <- BIC(rss_reg,n,k)
Reg_BIC
RightReg_BIC <- BIC(rss_Rreg,n,k)
RightReg_BIC

[1] 188.3073

[1] 66.15489

[1] 100.2345

[1] 108.914

[1] "variability"


...

[1] -30.26221

[1] "ACF measures"


[1] 118.0406

[1] -44.41664

[1] -65.27983